# Import Libraries

In [ ]:
#!pip install pygal_maps_world
#import pygal
#from pygal_maps_world.maps import World


import numpy as np # linear algebra
from numpy import zeros
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.linear_model import LogisticRegression

import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# Load Data and review

In [ ]:
data_ks = pd.read_csv("../input/kaggle-survey-2020/kaggle_survey_2020_responses.csv")
country_Code = pd.read_csv("../input/countrycode/country.csv")

In [ ]:
data_ks.head()

# Drop first row which is just the questions

In [ ]:
data_ks = data_ks.drop(0)

# Grouping data by age and country

In [ ]:
data_ks1 = data_ks.groupby(['Q3','Q1'])['Q1'].count()
data_ks1 = data_ks1.unstack()
data_ks1 = data_ks1.fillna(0)
data_ks1["sum"] = data_ks1.sum(axis=1)
data_ks1 = data_ks1.sort_values(by=['sum'], ascending=False)

In [ ]:
data_ks1.columns

# Age Distribution across multiple countries

In [ ]:
data_ks2 = data_ks1[['18-21', '22-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54',
       '55-59', '60-69', '70+']]

data_ks2 = data_ks2[:10]
data_ks2.plot(figsize=(20,20), kind='bar', stacked=True, title = 'Top 10 countries with Age Distribution')
plt.show()

India clearly has taken the lead. Both in the number of engineers as well as the number of engineers at lower age band. This would help ensure that in the future there are more experience ML engineers in India 

In [ ]:
data_ks1.head()

# Age Percentage Distribution across multiple countries

In [ ]:
data_ks3 = data_ks1[['18-21', '22-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54',
       '55-59', '60-69', '70+']].div(data_ks1['sum'].values,axis=0)

In [ ]:
data_ks3.head()

In [ ]:
data_ks4 = data_ks3
plt.figure(figsize = (16,16))
ax = sns.heatmap(data_ks4,annot=True)
ax.set_title('Age Percentage Distribution across multiple countries')

In [ ]:
data_ks4 = data_ks3[:15]
plt.figure(figsize = (16,16))
ax = sns.heatmap(data_ks4,annot=True)
ax.set_title('Age Percentage Distribution across top 10 countries')

In [ ]:
print(data_ks1.index)

In [ ]:
from numpy import array

data_ks5 = pd.DataFrame(data_ks1['sum'])
data5 = np.zeros(country_Code.shape[0])

for index, row in country_Code.iterrows():
    for index1, row1 in data_ks5.iterrows():
        if (index1 == row['Country']):
            data5[index] = row1['sum']       
            
country_Code['Count'] = pd.Series(data5, index=country_Code.index)

In [ ]:
#worldmap = World()

#data_dict = country_Code[['Symbol','Count']].set_index('Symbol').to_dict()
    
# set the title of the map 
#worldmap.title = 'Countries'

# adding the countries 
#worldmap.add('Engineer Count',data_dict['Count']) 
  
# save into the file 
#worldmap.render_to_file('abc.svg')

In [ ]:
country_Code

# Distribution representation Country wise

This method of representation I got from Gabriel Preda (gpreda)

Link to his notebook:
https://www.kaggle.com/gpreda/data-scientists-in-2020-kaggle-survey


In [ ]:
trace = go.Choropleth(
            locations = country_Code['Country'],
            locationmode='country names',
            z = country_Code['Count'],
            text = country_Code['Country'],
            autocolorscale =False,
            reversescale = True,
            colorscale = 'rainbow',
            marker = dict(
                line = dict(
                    color = 'rgb(0,0,0)',
                    width = 0.5)
            ),
            colorbar = dict(
                title = 'Responses from each country',
                tickprefix = '')
        )

data = [trace]
layout = go.Layout(
    title = 'Repesenttaion of respondents in each country',
    autosize=True,
    width=1000,
    height=1000,
    geo = dict(
        showframe = True,
        showlakes = False,
        showcoastlines = True,
        projection = dict(
            type = 'natural earth'
        )
    )
)

fig = dict( data=data, layout=layout )
iplot(fig)

# Grouping by gender and country

In [ ]:
data_ks1 = data_ks.groupby(['Q3','Q2'])['Q2'].count()
data_ks1 = data_ks1.unstack()
data_ks1 = data_ks1.fillna(0)
data_ks1["sum"] = data_ks1.sum(axis=1)
data_ks1 = data_ks1.sort_values(by=['sum'], ascending=False)
data_ks1 = data_ks1[:15]

In [ ]:
data_ks1.columns

In [ ]:
data_ks3 = data_ks1[['Man', 'Woman']].div(data_ks1['sum'].values,axis=0)

In [ ]:
data_ks4 = data_ks3
data_ks4 = data_ks4.sort_values(by=['Woman'], ascending=False)
plt.figure(figsize = (16,16))
ax = sns.heatmap(data_ks4,annot=True)
ax.set_title('Gender Percentage Distribution across multiple countries')

# Grouping by Highest education and country

In [ ]:
data_ks1 = data_ks.groupby(['Q3','Q5'])['Q5'].count()
data_ks1 = data_ks1.unstack()
data_ks1 = data_ks1.fillna(0)
column_names = data_ks1.columns
data_ks1["sum"] = data_ks1.sum(axis=1)
data_ks1 = data_ks1.sort_values(by=['sum'], ascending=False)
data_ks1 = data_ks1[:15]
data_ks3 = data_ks1[column_names].div(data_ks1['sum'].values,axis=0)

In [ ]:
plt.figure(figsize = (16,16))
ax = sns.heatmap(data_ks3,annot=True)
ax.set_title('Highest Education Percentage Distribution across top 10 countries')

# Analysing India and US data for Years of Experience

Data screened for US and India and ignoring 'currently not employed' people

In [ ]:
data_ks5 = data_ks
data_ks5['Q6'] = data_ks5['Q6'].replace(['5-10 years', '10-20 years', '3-5 years', '< 1 years', '1-2 years',
       '20+ years'],['10','20','05','01','02','20+'])
data_ks5['Q15'] = data_ks5['Q15'].replace(['1-2 years', 'I do not use machine learning methods', '3-4 years',
       'Under 1 year', '2-3 years', '4-5 years', '5-10 years',
       '20 or more years', '10-20 years'],['02','00','04','01','03','05','10','20+','20'])

details_World = data_ks5[(data_ks['Q5'] != 'Currently not employed')][['Q3','Q5','Q6','Q15']].dropna()
details_US = data_ks5[((data_ks['Q5'] != 'Currently not employed') & (data_ks['Q3'] == 'United States of America'))][['Q3','Q5','Q6','Q15']].dropna()
details_Ind = data_ks5[((data_ks['Q5'] != 'Currently not employed') & (data_ks['Q3'] == 'India'))][['Q3','Q5','Q6','Q15']].dropna()

profession = details_World.groupby('Q5').sum().index.values
code_exp = details_World.groupby('Q6').sum().index.values
machine_exp = details_World.groupby('Q15').sum().index.values

In [ ]:
data_ks['Q15'].unique()

In [ ]:
details_Ind

# India Student Profile

In [ ]:
details_i = details_Ind[(details_Ind['Q5'] == 'Student')]

details_i=details_i.sort_values(by=['Q6','Q15']).groupby(['Q6','Q15']).count()
details_i=details_i.reset_index().pivot('Q6','Q15','Q5').fillna(0).astype('int')
details_i.head()
sns.color_palette("cubehelix")
ax = sns.heatmap(details_i)


In India the student population is new to programming. 
They have on an average 2 - 5 years of programming knowledge and 1-2 years of machine learning knowledge

# US Student Profile

In [ ]:
details_i = details_US[(details_US['Q5'] == 'Student')]

details_i=details_i.sort_values(by=['Q6','Q15']).groupby(['Q6','Q15']).count()
details_i=details_i.reset_index().pivot('Q6','Q15','Q5').fillna(0).astype('int')
details_i.head()
sns.color_palette("cubehelix")
ax = sns.heatmap(details_i)

In US the student population has higher programming knowledge. 
They have on an average 5-10 years of programming knowledge and 3 years of machine learning knowledge

* This shows that profiles of people vary from country to country.
* More advanaced countries will have people in each profile with higher experience

# Comparing Engineer profile between World, US and India

In [ ]:
fig, ax = plt.subplots(len(profession),3, figsize=(18,20), dpi=100)
i = 0

details_type = details_World[(details_World['Q5'] == 'Business Analyst')]
details_type = details_type.sort_values(by=['Q6','Q15']).groupby(['Q6','Q15']).count()
details_type = details_type.reset_index().pivot('Q6','Q15','Q5').fillna(0).astype('int')

fig.suptitle('World                                               US                                               India', fontsize=20)

#ax[0][0].set_title(str("World"), rotation=0)
#ax[0][1].set_title(str("US"), rotation=0)
#ax[0][2].set_title(str("India"), rotation=0)
        
for prof in profession:
    details_iWorld = details_World[(details_World['Q5'] == prof)]
    details_iWorld=details_iWorld.sort_values(by=['Q6','Q15']).groupby(['Q6','Q15']).count()
    details_iWorld=details_iWorld.reset_index().pivot('Q6','Q15','Q5').fillna(0).astype('int')
    
    details_iUS = details_US[(details_US['Q5'] == prof)]
    details_iUS=details_iUS.sort_values(by=['Q6','Q15']).groupby(['Q6','Q15']).count()
    details_iUS=details_iUS.reset_index().pivot('Q6','Q15','Q5').fillna(0).astype('int')

    details_iInd = details_Ind[(details_Ind['Q5'] == prof)]
    details_iInd=details_iInd.sort_values(by=['Q6','Q15']).groupby(['Q6','Q15']).count()
    details_iInd=details_iInd.reset_index().pivot('Q6','Q15','Q5').fillna(0).astype('int')

    
    ax[i][0].matshow(details_iWorld, cmap='GnBu')
    ax[i][0].set_xticks(np.arange(len(details_type.columns)))
    ax[i][0].set_yticks(np.arange(len(details_type.index)))
    ax[i][0].grid(False)
    ax[i][0].set_title(prof, y=-0.1, rotation=0)


    ax[i][1].matshow(details_iUS, cmap='GnBu')
    ax[i][1].set_xticks(np.arange(len(details_type.columns)))
    ax[i][1].set_yticks(np.arange(len(details_type.index)))
    ax[i][1].grid(False)
    
   
    ax[i][2].matshow(details_iInd, cmap='GnBu')
    ax[i][2].set_xticks(np.arange(len(details_type.columns)))
    ax[i][2].set_yticks(np.arange(len(details_type.index)))
    ax[i][2].grid(False)
    
    ax[i][0].set_xticklabels([])
    ax[i][0].set_yticklabels(details_type.index)

    if i == 0:
        ax[i][0].set_xticklabels(details_type.columns, rotation=90)
        ax[i][1].set_xticklabels(details_type.columns, rotation=90)
        ax[i][1].set_yticklabels([])
        ax[i][2].set_xticklabels(details_type.columns, rotation=90)
        ax[i][2].set_yticklabels([])

    else:
        ax[i][1].set_xticklabels([])
        ax[i][1].set_yticklabels([])
        ax[i][2].set_xticklabels([])
        ax[i][2].set_yticklabels([])

    i= i + 1
    
plt.savefig('ProfileCompare.png')
plt.show()  
    


# Y axis is Coding experience and X axis is Machine Learning Experience

It is clear that in US in each designation the engineers carry higher experience in both Coding experience and Machine Learning experience

# Analysing India and US data based on Companies

In [ ]:
data_ks6 = data_ks


data_ks6['Q20'] = data_ks6['Q20'].replace(['10,000 or more employees', '1000-9,999 employees',
       '250-999 employees', '0-49 employees', '50-249 employees'],['10K+','10K','01K','00050','00250'])
data_ks5['Q21'] = data_ks5['Q21'].replace(['20+', '0', '5-9', '1-2', '3-4', '10-14', '15-19'],['20+','00','09','02','04','14','19'])

details_World = data_ks5[(data_ks['Q5'] != 'Currently not employed')][['Q3','Q5','Q20','Q21']].dropna()
details_US = data_ks5[((data_ks['Q5'] != 'Currently not employed') & (data_ks['Q3'] == 'United States of America'))][['Q3','Q5','Q20','Q21']].dropna()
details_Ind = data_ks5[((data_ks['Q5'] != 'Currently not employed') & (data_ks['Q3'] == 'India'))][['Q3','Q5','Q20','Q21']].dropna()

profession = details_World.groupby('Q5').sum().index.values

In [ ]:
fig, ax = plt.subplots(len(profession),3, figsize=(18,20), dpi=100)
i = 0

details_type = details_World[(details_World['Q5'] == 'Business Analyst')]
details_type = details_type.sort_values(by=['Q20','Q21']).groupby(['Q20','Q21']).count()
details_type = details_type.reset_index().pivot('Q20','Q21','Q5').fillna(0).astype('int')

fig.suptitle('World                                               US                                               India', fontsize=20)

#ax[0][0].set_title(str("World"), rotation=0)
#ax[0][1].set_title(str("US"), rotation=0)
#ax[0][2].set_title(str("India"), rotation=0)
        
for prof in profession:
    details_iWorld = details_World[(details_World['Q5'] == prof)]
    details_iWorld=details_iWorld.sort_values(by=['Q20','Q21']).groupby(['Q20','Q21']).count()
    details_iWorld=details_iWorld.reset_index().pivot('Q20','Q21','Q5').fillna(0).astype('int')
    
    details_iUS = details_US[(details_US['Q5'] == prof)]
    details_iUS=details_iUS.sort_values(by=['Q20','Q21']).groupby(['Q20','Q21']).count()
    details_iUS=details_iUS.reset_index().pivot('Q20','Q21','Q5').fillna(0).astype('int')

    details_iInd = details_Ind[(details_Ind['Q5'] == prof)]
    details_iInd=details_iInd.sort_values(by=['Q20','Q21']).groupby(['Q20','Q21']).count()
    details_iInd=details_iInd.reset_index().pivot('Q20','Q21','Q5').fillna(0).astype('int')

    
    ax[i][0].matshow(details_iWorld, cmap='GnBu')
    ax[i][0].set_xticks(np.arange(len(details_type.columns)))
    ax[i][0].set_yticks(np.arange(len(details_type.index)))
    ax[i][0].grid(False)
    ax[i][0].set_title(prof, y=-0.1, rotation=0)


    ax[i][1].matshow(details_iUS, cmap='GnBu')
    ax[i][1].set_xticks(np.arange(len(details_type.columns)))
    ax[i][1].set_yticks(np.arange(len(details_type.index)))
    ax[i][1].grid(False)
    
   
    ax[i][2].matshow(details_iInd, cmap='GnBu')
    ax[i][2].set_xticks(np.arange(len(details_type.columns)))
    ax[i][2].set_yticks(np.arange(len(details_type.index)))
    ax[i][2].grid(False)
    
    ax[i][0].set_xticklabels([])
    ax[i][0].set_yticklabels(details_type.index)

    if i == 0:
        ax[i][0].set_xticklabels(details_type.columns, rotation=90)
        ax[i][1].set_xticklabels(details_type.columns, rotation=90)
        ax[i][1].set_yticklabels([])
        ax[i][2].set_xticklabels(details_type.columns, rotation=90)
        ax[i][2].set_yticklabels([])

    else:
        ax[i][1].set_xticklabels([])
        ax[i][1].set_yticklabels([])
        ax[i][2].set_xticklabels([])
        ax[i][2].set_yticklabels([])

    i= i + 1
    
plt.savefig('CompanyCompare.png')
plt.show()  
    

# Y axis is Size of Company and X axis is Size of Team

If you look Datat Scientist, ML engineers and Statiticians in US work for large companies but in India such people work for smaller startup companies. In India large IT companies are primarily IT outsourcing companies which work with US clients and help deploy ML solutions with lesser experienced Data Science engineers who do more depolyment and customization and less work on developing new algorithm. The Datat Scientist, ML engineers and Statiticians in India are primarily work in startups where they are trying to build breakthrough products.

In [ ]:
details_World.head()

# Countries having higher percetage of Woman Data Experts 

In [ ]:
import pandas as pd
import plotly.express as px

data_ks6 = data_ks4.reset_index()

xmin, xmax = min(data_ks6.Man), max(data_ks6.Man)
ymin, ymax = min(data_ks6.Woman), max(data_ks6.Woman)

fig = px.scatter(data_ks6, x="Man", y="Woman", color="Q3", hover_name="Q3",facet_col="Q3", width=3000, height=400, log_x=True, size_max=45,range_x=[xmin,xmax], range_y=[ymin,ymax])

fig.show()

In [ ]:
data_ks3 = data_ks1.reset_index()

data_ks3['sum_niche'] =  data_ks3['Data Scientist'] + data_ks3['Research Scientist'] + data_ks3['Statistician']

# Countries with Higher percentage of Niche profiles

In [ ]:
xmin, xmax = min(data_ks3["sum"]), max(data_ks3["sum"])

ymin, ymax = min(data_ks3["sum_niche"]), max(data_ks3["sum_niche"])

fig = px.scatter(data_ks3, x="sum", y="sum_niche", color="Q3", hover_name="Q3",facet_col="Q3", width=3000, height=400, log_x=True, size_max=45,range_x=[xmin,xmax], range_y=[ymin,ymax])

fig.show()

References:

https://www.kaggle.com/uyangas/becoming-a-data-scientist

https://www.kaggle.com/gpreda/data-scientists-in-2020-kaggle-survey
